In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
#import chart_studio.plotly as py
import plotly.express as px
from datetime import datetime

In [6]:
paths = ["../probes.db", "../../../data.db"]
data = None
for path in paths:
    if os.path.exists(path):
        with sqlite3.connect(path) as connection:
            c = connection.cursor()
            c.execute("select * from 'probe-request' limit 500000")
            d = c.fetchall()
            data = []
            for vals in d:
                vals = list(vals)
                vals[0] = datetime.fromisoformat(vals[0])
                data.append(vals)
        break

In [7]:
if data:
    data = pd.DataFrame(data, columns=['datetime', 'mac', 'make', 'ssid', 'ssid_caps', 'dbm'])

In [8]:
data

,datetime,mac,make,ssid,ssid_caps,dbm
0,2020-03-23 12:21:59.531987,80:19:34:ac:7d:17,Intel Corporate,,,-20.0
1,2020-03-30 12:10:58.450316,d2:3a:a0:a0:fd:2a,UNKNOWN,,,-70.0
2,2020-03-30 12:11:25.636282,0c:70:4a:ff:52:91,UNKNOWN,,,-83.0
3,2020-03-30 12:11:26.087794,0c:70:4a:ff:52:91,UNKNOWN,,,-80.0
4,2020-03-30 12:12:45.368070,1c:91:48:98:88:4a,"Apple, Inc.",,,-87.0
...,...,...,...,...,...,...
499995,2020-06-22 16:59:24.350042,2a:cf:5e:c6:de:0c,UNKNOWN,,,-57.0
499996,2020-06-22 16:59:24.494246,2a:cf:5e:c6:de:0c,UNKNOWN,,,-56.0
499997,2020-06-22 16:59:27.536261,b8:90:47:1a:31:f4,UNKNOWN,,,-64.0
499998,2020-06-22 16:59:27.641724,b8:90:47:1a:31:f4,UNKNOWN,,,-65.0


In [9]:
my_mac = 'e4:0e:ee:**:**:**'

In [14]:
my_data = data[data.mac == my_mac]

In [15]:
min, max = my_data.datetime.min(), my_data.datetime.max()

In [17]:
range = pd.date_range(min, max)

In [18]:
# Meine mac zeit werte boolean über range plotten (da/nicht da)

DatetimeIndex(['2020-03-30 12:49:29.388366', '2020-03-31 12:49:29.388366',
               '2020-04-01 12:49:29.388366', '2020-04-02 12:49:29.388366',
               '2020-04-03 12:49:29.388366', '2020-04-04 12:49:29.388366',
               '2020-04-05 12:49:29.388366', '2020-04-06 12:49:29.388366',
               '2020-04-07 12:49:29.388366', '2020-04-08 12:49:29.388366',
               '2020-04-09 12:49:29.388366', '2020-04-10 12:49:29.388366',
               '2020-04-11 12:49:29.388366', '2020-04-12 12:49:29.388366',
               '2020-04-13 12:49:29.388366', '2020-04-14 12:49:29.388366',
               '2020-04-15 12:49:29.388366', '2020-04-16 12:49:29.388366',
               '2020-04-17 12:49:29.388366', '2020-04-18 12:49:29.388366',
               '2020-04-19 12:49:29.388366', '2020-04-20 12:49:29.388366',
               '2020-04-21 12:49:29.388366', '2020-04-22 12:49:29.388366',
               '2020-04-23 12:49:29.388366', '2020-04-24 12:49:29.388366',
               '2020-04-2

In [ ]:
len(data.macaddress.unique())

In [ ]:
anonymized = data.loc[data.macaddress.str.contains(r'1\d$', regex=True)]

In [ ]:
clear = data.loc[~data.macaddress.str.contains(r'1\d$', regex=True)]

In [ ]:
px.scatter(data, x='datetime', y='rssi', color='make')

In [ ]:
yesterday=data.loc[(data['datetime'] > datetime(2020,5,8)) & (data['datetime'] <= datetime(2020,5,9))]

In [ ]:
px.scatter(yesterday, x='macaddress', y='rssi', color='make', width=1500)

In [ ]:
yesterday_12h = data[(data.datetime.dt.day == 8) & (data.datetime.dt.month == 5) & (data.datetime.dt.hour == 12)]
yesterday.head()

In [ ]:
burst = pd.DataFrame()
burst['mac'] = yesterday_12h.macaddress.unique()
burst.head()

In [ ]:
for dt, mac, _, _, _, _ in yesterday_12h.values:
    print(dt.minute, mac)
    break

In [ ]:
min_burst={}

for mac in yesterday_12h.macaddress.unique():
        min_burst.update({mac: {i: 0 for i in range(0, 60)}})



In [ ]:
for dt, mac, _, _, _, _ in yesterday_12h.values:
    min_burst[mac][dt.minute] += 1

In [ ]:
mb={i: {} for i in range(0, 60)}
for dt, mac, _, _, _, _ in yesterday_12h.values:
    if mac in mb[dt.minute].keys():
        mb[dt.minute][mac] += 1
    else:
        mb[dt.minute].update({mac: 1})
mb
dfmb = pd.DataFrame(mb).fillna(0)
dfmb.head()

In [ ]:
bursts = pd.DataFrame(min_burst, index=range(len(yesterday_12h.macaddress.unique())))

bursts.head(2)

In [ ]:
bursts.columns

In [ ]:
import plotly.graph_objects as go
from plotly.colors import n_colors


fig = go.Figure(data=[go.Table(
  cells=dict(values=[bursts[column] for column in bursts.columns],
    #line_color=[np.array(colors)[a],np.array(colors)[b], np.array(colors)[c]],
    #fill_color=[np.array(colors)[a],np.array(colors)[b], np.array(colors)[c]],
    ))
])

fig.show()

In [ ]:
clear.macaddress.unique()

In [ ]:
macs_count = data.macaddress.value_counts()
macs_count.values

In [ ]:
px.line(x=macs_count.index, y=macs_count.values)

In [ ]:
dev_1 = data[data.macaddress == '80:19:34:ac:7d:17']

In [ ]:
dev_1

In [ ]:
px.scatter(dev_1, 'datetime', 'rssi')

In [ ]:
px.scatter(dev_1[(dev_1.datetime.dt.day == 7) & (dev_1.datetime.dt.month == 5)], 'datetime', 'rssi')